In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import shap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sn
from sklearn.decomposition import PCA 
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,BatchNormalization,Dropout,SimpleRNN,GRU
import tensorflow as tf
from sklearn.ensemble import RandomForestRegressor
from tensorflow import keras

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import shap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sn
from sklearn.decomposition import PCA 
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,BatchNormalization,Dropout
import tensorflow as tf
from sklearn.ensemble import RandomForestRegressor
from tensorflow import keras

In [3]:
df = pd.read_excel("the total emissions data from three carbon-emitting sectors.xlsx")
df = df.drop(columns=['日期','iso_code','国家'])
X = df[100:]
y=X['carbon emission']
X = X.drop(columns=['carbon emission'])
X = X.drop(columns=['daily_people_vaccinated_per_hundred'])
#X = X.dropna() 

In [ ]:

def create_dataset(data, lookback=1):
    X, y = [], []
    for i in range(len(data)-lookback):
        X.append(data[i:i+lookback, :])
        y.append(data[i+lookback, 0])
    return np.array(X), np.array(y)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(np.array(y).reshape(-1,1))
X_time,y_time = create_dataset(scaled_data,20)
train_x,test_x,train_y,test_y = train_test_split(X_time,y_time)
#GRU
batch_size = 1000  
steps = 50  
n_features = 1  
n_outputs = 1  
epochs = 10  
model = Sequential()  
model.add(GRU(10, input_shape=(20, n_features)))  
#model.add(GRU(50))  
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(train_x, train_y, epochs=10, batch_size=1, verbose=2,validation_data=(test_x,test_y))
ttest_x = model.predict(test_x)
test_predict = scaler.inverse_transform(ttest_x)
Y_rel = scaler.inverse_transform(np.array(test_y).reshape(-1,1))
print(((test_predict-Y_rel)**2).mean())
print(abs((test_predict-Y_rel)).mean())
ss_res = ((test_predict - Y_rel) ** 2).sum()  
ss_tot = ((Y_rel - Y_rel.mean()) ** 2).sum()  
r2 = 1 - (ss_res / ss_tot)  
print(f'R-squared: {r2.item():.4f}')

In [4]:
def create_dataset(data, lookback=1):
    X, y = [], []
    for i in range(len(data)-lookback):
        X.append(data[i:i+lookback, :])
        y.append(data[i+lookback, 0])
    return np.array(X), np.array(y)

In [20]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(np.array(y).reshape(-1,1))
X_time,y_time = create_dataset(scaled_data,20)

In [36]:
train_x,test_x,train_y,test_y = train_test_split(X_time,y_time)

In [30]:
#GRU
batch_size = 1000  
steps = 50  
n_features = 1  
n_outputs = 1  
epochs = 10  
model = Sequential()  
model.add(GRU(10, input_shape=(20, n_features)))  
#model.add(GRU(50))  
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(train_x, train_y, epochs=10, batch_size=1, verbose=2,validation_data=(test_x,test_y))
ttest_x = model.predict(test_x)
test_predict = scaler.inverse_transform(ttest_x)
Y_rel = scaler.inverse_transform(np.array(test_y).reshape(-1,1))
print(((test_predict-Y_rel)**2).mean())
print(abs((test_predict-Y_rel)).mean())
ss_res = ((test_predict - Y_rel) ** 2).sum()  
ss_tot = ((Y_rel - Y_rel.mean()) ** 2).sum()  
r2 = 1 - (ss_res / ss_tot)  
print(f'R-squared: {r2.item():.4f}')

Epoch 1/10
462/462 - 2s - loss: 0.0093 - val_loss: 0.0063 - 2s/epoch - 4ms/step
Epoch 2/10
462/462 - 1s - loss: 0.0080 - val_loss: 0.0078 - 1s/epoch - 2ms/step
Epoch 3/10
462/462 - 1s - loss: 0.0077 - val_loss: 0.0062 - 1s/epoch - 2ms/step
Epoch 4/10
462/462 - 1s - loss: 0.0077 - val_loss: 0.0063 - 1s/epoch - 2ms/step
Epoch 5/10
462/462 - 1s - loss: 0.0074 - val_loss: 0.0061 - 1s/epoch - 2ms/step
Epoch 6/10
462/462 - 1s - loss: 0.0075 - val_loss: 0.0064 - 1s/epoch - 2ms/step
Epoch 7/10
462/462 - 1s - loss: 0.0073 - val_loss: 0.0061 - 1s/epoch - 2ms/step
Epoch 8/10
462/462 - 1s - loss: 0.0074 - val_loss: 0.0063 - 1s/epoch - 2ms/step
Epoch 9/10
462/462 - 1s - loss: 0.0071 - val_loss: 0.0067 - 1s/epoch - 2ms/step
Epoch 10/10
462/462 - 1s - loss: 0.0072 - val_loss: 0.0061 - 1s/epoch - 2ms/step
5/5 [==============================] - 0s 2ms/step
0.318823567342
0.51034325678
R-squared: 0.7471


In [28]:
#LSTM
model = Sequential()
model.add(LSTM(200,input_shape=(20, 1)))
#model.add(LSTM(100))return_sequences=True,
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(train_x, train_y, epochs=10, batch_size=1, verbose=2,validation_data=(test_x,test_y))
ttest_x = model.predict(test_x)
test_predict = scaler.inverse_transform(ttest_x)
Y_rel = scaler.inverse_transform(np.array(test_y).reshape(-1,1))
print(((test_predict-Y_rel)**2).mean())
print(abs((test_predict-Y_rel)).mean())
ss_res = ((test_predict - Y_rel) ** 2).sum()  
ss_tot = ((Y_rel - Y_rel.mean()) ** 2).sum()  
r2 = 1 - (ss_res / ss_tot)  
print(f'R-squared: {r2.item():.4f}')

Epoch 1/10
462/462 - 3s - loss: 0.0168 - val_loss: 0.0079 - 3s/epoch - 6ms/step
Epoch 2/10
462/462 - 2s - loss: 0.0107 - val_loss: 0.0071 - 2s/epoch - 4ms/step
Epoch 3/10
462/462 - 2s - loss: 0.0093 - val_loss: 0.0068 - 2s/epoch - 4ms/step
Epoch 4/10
462/462 - 2s - loss: 0.0082 - val_loss: 0.0069 - 2s/epoch - 4ms/step
Epoch 5/10
462/462 - 2s - loss: 0.0078 - val_loss: 0.0060 - 2s/epoch - 4ms/step
Epoch 6/10
462/462 - 2s - loss: 0.0079 - val_loss: 0.0067 - 2s/epoch - 4ms/step
Epoch 7/10
462/462 - 2s - loss: 0.0080 - val_loss: 0.0066 - 2s/epoch - 4ms/step
Epoch 8/10
462/462 - 2s - loss: 0.0076 - val_loss: 0.0061 - 2s/epoch - 4ms/step
Epoch 9/10
462/462 - 2s - loss: 0.0079 - val_loss: 0.0063 - 2s/epoch - 4ms/step
Epoch 10/10
462/462 - 2s - loss: 0.0077 - val_loss: 0.0068 - 2s/epoch - 4ms/step
5/5 [==============================] - 0s 5ms/step
0.2836121260478416
0.38658046084872155
R-squared: 0.7814


In [39]:
#RNN
model = Sequential()
model.add(SimpleRNN(50,input_shape=(20, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(train_x, train_y, epochs=10, batch_size=1, verbose=2,validation_data=(test_x,test_y))
ttest_x = model.predict(test_x)
test_predict = scaler.inverse_transform(ttest_x)
Y_rel = scaler.inverse_transform(np.array(test_y).reshape(-1,1))
print(((test_predict-Y_rel)**2).mean())
print(abs((test_predict-Y_rel)).mean())
ss_res = ((test_predict - Y_rel) ** 2).sum()  
ss_tot = ((Y_rel - Y_rel.mean()) ** 2).sum()  
r2 = 1 - (ss_res / ss_tot)  
print(f'R-squared: {r2.item():.4f}')

Epoch 1/10
462/462 - 1s - loss: 2.3098 - val_loss: 0.4888 - 1s/epoch - 2ms/step
Epoch 2/10
462/462 - 1s - loss: 0.4199 - val_loss: 0.3281 - 647ms/epoch - 1ms/step
Epoch 3/10
462/462 - 1s - loss: 0.3353 - val_loss: 0.3483 - 656ms/epoch - 1ms/step
Epoch 4/10
462/462 - 1s - loss: 0.3236 - val_loss: 0.4016 - 623ms/epoch - 1ms/step
Epoch 5/10
462/462 - 1s - loss: 0.3342 - val_loss: 0.3417 - 649ms/epoch - 1ms/step
Epoch 6/10
462/462 - 1s - loss: 0.3231 - val_loss: 0.3846 - 631ms/epoch - 1ms/step
Epoch 7/10
462/462 - 1s - loss: 0.3054 - val_loss: 0.5025 - 636ms/epoch - 1ms/step
Epoch 8/10
462/462 - 1s - loss: 0.3009 - val_loss: 0.4095 - 643ms/epoch - 1ms/step
Epoch 9/10
462/462 - 1s - loss: 0.3094 - val_loss: 0.3435 - 644ms/epoch - 1ms/step
Epoch 10/10
462/462 - 1s - loss: 0.3087 - val_loss: 0.3342 - 657ms/epoch - 1ms/step
5/5 [==============================] - 0s 1ms/step
0.4644325556
0.5201242345
R-squared: 0.6687
